In [1]:
import numpy as np
import cv2 as cv
import os
import matplotlib as plt

In [2]:
def FindFormat(path, fileFormat):
    """Recursively walks through folders and files in path looking for .<fileFormat> -formatted files"""
    lPathes = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(fileFormat):
                lPathes.append(os.path.join(root, file))
    return lPathes

In [3]:
path_YES = os.path.abspath(os.path.join("signals_dataset", "YES(fits)"))
path_NO = os.path.abspath(os.path.join("signals_dataset", "NO(doesn_t_fit)"))

In [4]:
def find_contour_proportions(filename: str) -> float:
    image = cv.imread(filename)
    raw_image = image.copy()

    bw_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blurred_image = cv.GaussianBlur(bw_image, (3, 3), 0)
    _, otsu_image = cv.threshold(blurred_image, 0, 255, cv.THRESH_OTSU)

    kernel = np.ones((8, 8), np.uint8)
    opened_image = cv.morphologyEx(otsu_image, cv.MORPH_OPEN, kernel)
    closed_image = cv.morphologyEx(opened_image, cv.MORPH_CLOSE, kernel)

    contours, _ = cv.findContours(closed_image, mode=cv.RETR_LIST, method=cv.CHAIN_APPROX_SIMPLE)
    sorted_contours = sorted(contours, key=lambda x: cv.contourArea(x), reverse=True)
    big_contours = sorted_contours[2:3]

    contours_image = raw_image.copy()
    for c in big_contours:
        x, y, w, h = cv.boundingRect(c)
        cv.rectangle(contours_image, (x, y), (x + w, y + h), (0, 255, 0), thickness=15)

    return w / h

Надо получить границу этим же методом. Для этого передадим в метод граничный случай.

In [5]:
threshold = find_contour_proportions('D:\\current_work\\!V KURS\\8_sem\\SIGNALS\\Maxi_Laba\\signals_dataset\\YES(fits)\\12.jpg')

In [6]:
threshold

0.5770975056689343

In [7]:
filenames = FindFormat(path_YES, ".jpg")
true = 0
false = 0
for filename in filenames:
    ratio = find_contour_proportions(filename)
    if ratio <= threshold:
        print(os.path.basename(filename), "YES")
        true += 1
    else:
        print(os.path.basename(filename), "NO")
        false += 1
        
filenames = FindFormat(path_NO, ".jpg")
for filename in filenames:
    ratio = find_contour_proportions(filename)
    if ratio > threshold:
        print(os.path.basename(filename), "NO")
        true += 1
    else:
        print(os.path.basename(filename), "YES")
        false += 1
print(true/(true + false))

1.jpg NO
10.jpg NO
11.jpg NO
12.jpg YES
13.jpg YES
14.jpg YES
15.jpg YES
2.jpg NO
3.jpg NO
4.jpg NO
5.jpg YES
6.jpg NO
7.jpg YES
8.jpg NO
9.jpg YES
1.jpg NO
10.jpg NO
11.jpg NO
12.jpg NO
13.jpg YES
14.jpg YES
15.jpg NO
2.jpg NO
3.jpg NO
4.jpg NO
5.jpg YES
6.jpg NO
7.jpg NO
8.jpg NO
9.jpg NO
0.6333333333333333
